In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import pandas as pd
import time
from langchain_chroma import Chroma
from sentence_transformers import SentenceTransformer
from langchain_core.documents import Document
from embedder import T5Embeddings
from db import get_db
from retreiver import GTR

In [3]:
df = pd.read_csv('/Users/ahmed/Desktop/msc-24/ECHR/paragraphs/echr_case_paragraphs_v2.csv')

In [4]:
db = get_db()

python3(8753) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


DB Loaded - Entries: 0


In [5]:
entires = db._collection.count()

print("Entries:", entires)


Entries: 0


In [6]:
all_paragraphs = [
    Document(
        page_content=f'{case_name}; § {paragraph_number}: {str(paragraph_text).replace(f"{paragraph_number}.", "").strip()}',
        metadata={
            "case_name": case_name,
            "paragraph_number": paragraph_number,
            "case_id": case_id,
            "date": date,
        },
    )
    for case_name, paragraph_number, paragraph_text, case_id, date in zip(
        df["case_name"], df["paragraph_number"], df["paragraph_text"], df["case_id"], df["date"]
    )
]


In [9]:
# Process paragraphs in batches to avoid memory issues
n = 2
while all_paragraphs:
    paragraphs = all_paragraphs[:n]
    all_paragraphs = all_paragraphs[n:]

    # Start timing
    start_time = time.time()

    # Add documents to Chroma
    db.add_documents(documents=paragraphs)

    # Stop timing
    end_time = time.time()

    elapsed_time = end_time - start_time
    minutes, seconds = divmod(elapsed_time, 60)

    print(f"Processed {len(paragraphs)} paragraphs ({int(minutes)}:{int(seconds)})")
    print(f"Remaining {len(all_paragraphs)} paragraphs")

KeyboardInterrupt: 